# PINN aplicado a la ecuacion de calor en dos dimensiones

## Librerias y configuracion

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
import imageio
import os

## Definimos parametros y funciones auxilaires

In [2]:
# Parámetros del problema
T = 1.0  # Tiempo final
Lx = 1.0  # Longitud del dominio en x
Ly = 1.0  # Longitud del dominio en y
nx = 50  # Número de puntos de la malla en x
ny = 50  # Número de puntos de la malla en y
nt = 100  # Número de pasos de tiempo
alpha = 0.1  # Coeficiente de difusión

# Funciones auxiliares
def u_exact(x, y, t):
    return np.exp(-2 * np.pi**2 * alpha * t) * np.sin(np.pi * x) * np.sin(np.pi * y)

def g(x, y):
    return np.zeros_like(x)

def f(x, y, t):
    return -4 * np.pi**2 * alpha * u_exact(x, y, t)


#### Creamos una malla y definimos la condicion inicial

In [3]:
# Crear la malla
x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny)
X, Y = np.meshgrid(x, y)

# Definir la condición inicial
u0 = np.sin(np.pi * X) * np.sin(np.pi * Y)


### Definimos el modelo y las funciones de perdida

In [12]:
# Definir la red neuronal
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(3, 20)
        self.fc2 = torch.nn.Linear(20, 20)
        self.fc3 = torch.nn.Linear(20, 20)
        self.fc4 = torch.nn.Linear(20, 1)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Definir la función de pérdida
def loss_fn(model, x, y, t):
    u_pred = model(torch.cat([x, y, t], dim=1))
    u_x = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u_pred, y, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
    f = u_t - (alpha*(u_xx + u_yy))
    u0_tensor = torch.tensor(u0, dtype=torch.float32, requires_grad=True).view(-1, 1).cuda()
    loss = torch.mean(torch.square(f)) + torch.mean(torch.square(u_pred[0: nx*ny, :] - u0_tensor))
    return loss

In [5]:
# Agregar los métodos g y f a la función de pérdida
loss_fn.g = staticmethod(g)
loss_fn.f = staticmethod(f)

In [6]:
# Crear el modelo, definir el optimizador y mover todo a la GPU
model = PINN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Definir la función de entrenamiento
def train(model, optimizer, num_epochs, u0, Lx, Ly, nt, alpha):
    losses = []
    for epoch in range(num_epochs):
        t = np.random.uniform(low=0, high=T, size=(1, 1))
        x = np.random.uniform(low=0, high=Lx, size=(nx, 1))
        y = np.random.uniform(low=0, high=Ly, size=(ny, 1))
        x_grid, y_grid = np.meshgrid(x, y)
        x_grid = torch.tensor(x_grid, dtype=torch.float32, requires_grad=True).cuda()
        y_grid = torch.tensor(y_grid, dtype=torch.float32, requires_grad=True).cuda()
        t = torch.tensor(t, dtype=torch.float32, requires_grad=True).cuda()
        # u0_tensor = torch.tensor(u0, dtype=torch.float32, requires_grad=True).cuda()
        u0_tensor = torch.tensor(u0, dtype=torch.float32, requires_grad=True).view(-1, 1).cuda()


        optimizer.zero_grad()
        loss = loss_fn(model, x_grid, y_grid, t) + torch.mean(torch.square(model(torch.cat([x_grid, y_grid, torch.zeros_like(t)], dim=1)) - u0_tensor))
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print("Epoch: {}/{} Loss: {:.6f}".format(epoch+1, num_epochs, loss.item()))

        # Guardar resultados
        if (epoch+1) % 10 == 0:
            x_plot = np.linspace(0, Lx, 100)
            y_plot = np.linspace(0, Ly, 100)
            X_plot, Y_plot = np.meshgrid(x_plot, y_plot)
            X_plot = torch.tensor(X_plot, dtype=torch.float32).cuda()
            Y_plot = torch.tensor(Y_plot, dtype=torch.float32).cuda()
            t_plot = torch.zeros_like(X_plot)
            u_plot = model(torch.cat([X_plot.reshape(-1, 1), Y_plot.reshape(-1, 1), t_plot.reshape(-1, 1)], dim=1)).detach().cpu().numpy().reshape(100, 100)
            u_exact_plot = u_exact(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), T)

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), u_plot, cmap='viridis')
            ax.set_title("PINN Solution at T = {:.2f}".format(T))
            plt.savefig("results/pinn_solution_epoch{}.png".format(epoch+1))
            plt.close()

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), u_exact_plot, cmap='viridis')
            ax.set_title("Exact Solution at T = {:.2f}".format(T))
            plt.savefig("results/exact_solution.png")
                # Crear animación
    images = []
    for i in range(10, num_epochs+1, 10):
        filename = "results/pinn_solution_epoch{}.png".format(i)
        images.append(imageio.imread(filename))
    imageio.mimsave("results/pinn_solution_animation.gif", images)

In [13]:
def train(model, optimizer, num_epochs, u0, Lx, Ly, nt, alpha):
    # Definir el tamaño de batch para el entrenamiento
    batch_size = 2000

    # Definir los grids de la malla
    x_grid = torch.linspace(0, Lx, 51)
    y_grid = torch.linspace(0, Ly, 51)
    t_grid = torch.linspace(0, nt*alpha, nt+1)

    # Convertir los grids a tensores y moverlos a la GPU
    x_grid = torch.tensor(x_grid, dtype=torch.float32, requires_grad=True).cuda()
    y_grid = torch.tensor(y_grid, dtype=torch.float32, requires_grad=True).cuda()
    t_grid = torch.tensor(t_grid, dtype=torch.float32, requires_grad=True).cuda()

    # Calcular el paso de tiempo
    dt = alpha*(Lx/50)**2

    # Definir el tensor de la condición inicial
    u0_tensor = torch.tensor(u0, dtype=torch.float32, requires_grad=True).cuda()

    # Entrenar el modelo
    for epoch in range(num_epochs):
        # Generar un batch aleatorio de puntos interiores
        x_int = torch.rand(batch_size, requires_grad=True).cuda()*Lx
        y_int = torch.rand(batch_size, requires_grad=True).cuda()*Ly
        t_int = torch.rand(batch_size, requires_grad=True).cuda()*nt*alpha

        # Calcular los valores predichos y las derivadas parciales del modelo
        u_pred = model(torch.cat([x_int[:,None], y_int[:,None], t_int[:,None]], dim=1))
        u_x = torch.autograd.grad(u_pred, x_int, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_y = torch.autograd.grad(u_pred, y_int, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x_int, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y_int, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]

        # Calcular la pérdida de la red
        loss = loss_fn(model, x_grid, y_grid, t_grid) + torch.mean(torch.square(u_xx + u_yy - 1/alpha * torch.autograd.grad(u_pred, t_int, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]))
        
        # Calcular la derivada de la pérdida y actualizar los pesos de la red
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Imprimir la pérdida cada 50 epochs
        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

        # Guardar resultados
        if (epoch+1) % 10 == 0:
            x_plot = np.linspace(0, Lx, 100)
            y_plot = np.linspace(0, Ly, 100)
            X_plot, Y_plot = np.meshgrid(x_plot, y_plot)
            X_plot = torch.tensor(X_plot, dtype=torch.float32).cuda()
            Y_plot = torch.tensor(Y_plot, dtype=torch.float32).cuda()
            t_plot = torch.zeros_like(X_plot)
            u_plot = model(torch.cat([X_plot.reshape(-1, 1), Y_plot.reshape(-1, 1), t_plot.reshape(-1, 1)], dim=1)).detach().cpu().numpy().reshape(100, 100)
            u_exact_plot = u_exact(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), T)

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), u_plot, cmap='viridis')
            ax.set_title("PINN Solution at T = {:.2f}".format(T))
            plt.savefig("results/pinn_solution_epoch{}.png".format(epoch+1))
            plt.close()

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X_plot.cpu().numpy(), Y_plot.cpu().numpy(), u_exact_plot, cmap='viridis')
            ax.set_title("Exact Solution at T = {:.2f}".format(T))
            plt.savefig("results/exact_solution.png")
                # Crear animación
    images = []
    for i in range(10, num_epochs+1, 10):
        filename = "results/pinn_solution_epoch{}.png".format(i)
        images.append(imageio.imread(filename))
    imageio.mimsave("results/pinn_solution_animation.gif", images)


In [21]:
def train(model, optimizer, num_epochs, u0, Lx, Ly, nt, alpha):
    # Definir los tensores x e y
    x_int = torch.linspace(0, Lx, 51, dtype=torch.float32).cuda()
    y_int = torch.linspace(0, Ly, 51, dtype=torch.float32).cuda()
    x_grid, y_grid = torch.meshgrid(x_int, y_int)
    x_grid, y_grid = x_grid.reshape(-1, 1), y_grid.reshape(-1, 1)
    
    # Definir el tensor t
    t_int = torch.linspace(0, 1, nt, dtype=torch.float32).reshape(nt, 1)
    t_grid = t_int.repeat(1, Lx*Ly).cuda()
    
    # Convertir u0 a tensor
    u0_tensor = torch.tensor(u0, dtype=torch.float32, requires_grad=True).cuda()
    
    # Entrenar el modelo
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        u_pred = model(torch.cat([x_grid, y_grid, t_grid], dim=2))
        
        u_x = torch.autograd.grad(u_pred, x_grid, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x_grid, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_y = torch.autograd.grad(u_pred, y_grid, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y_int, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
        
        loss = loss_fn(model, x_grid, y_grid, t_grid) + torch.mean(torch.square(u_xx + u_yy - 1/alpha * torch.autograd.grad(u_pred, t_int, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]))
        
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss {loss.item()}')
    
    # Guardar los resultados
    u_pred_np = u_pred.detach().cpu().numpy().reshape(nt, Lx, Ly)
    u0_np = u0_tensor.detach().cpu().numpy().reshape(Lx, Ly)
    np.save('results/u_pred', u_pred_np)
    np.save('results/u0', u0_np)
    
    # Graficar la solución
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    X, Y = np.meshgrid(x_int.cpu().numpy(), y_int.cpu().numpy())
    ax.plot_surface(X, Y, u_pred_np[0, :, :], cmap='coolwarm')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('u(x,y,0)')
    plt.savefig('results/solution.png')
    
    # # Crear animación
    # images = []
    # for i in range(nt):
    #     fig = plt.figure(figsize=(10, 6))
    #     ax = fig.add_subplot(1, 1, 1, projection='3d')
    #     ax.plot_surface(X, Y, u_pred_np[i, :, :], cmap='coolwarm')
    #     ax.set_xlabel('x')
    #     ax.set_ylabel('


In [22]:
# def train(model, optimizer, num_epochs, u0, Lx, Ly, nt, alpha):
#     # Preparar datos de entrenamiento
#     x_grid = torch.linspace(0, Lx, 51)
#     y_grid = torch.linspace(0, Ly, 51)
#     t_grid = torch.linspace(0, nt, 101)
#     x_int, y_int, t_int = torch.meshgrid(x_grid[1:-1], y_grid[1:-1], t_grid)

#     # Convertir datos de entrenamiento a tensores y enviarlos a la GPU
#     x_int = x_int.reshape(-1, 1).cuda()
#     y_int = y_int.reshape(-1, 1).cuda()
#     t_int = t_int.reshape(-1, 1).cuda()

#     # Definir la función de pérdida
#     def loss_fn(model, x, y, t):
#         u_pred = model(torch.cat([x, y, t], dim=1))
#         u_x = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
#         u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
#         u_y = torch.autograd.grad(u_pred, y, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]
#         u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
#         return torch.mean(torch.square(u_xx + u_yy - 1/alpha * torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(u_pred), create_graph=True)[0]))

#     # Entrenar la red neuronal
#     losses = []
#     for epoch in range(num_epochs):
#         optimizer.zero_grad()
#         loss = loss_fn(model, x_int, y_int, t_int) + torch.mean(torch.square(model(torch.cat([x_int, y_int, torch.zeros_like(t_int)], dim=1)) - u0_tensor))
#         loss.backward()
#         optimizer.step()

#         # Guardar el valor de la pérdida en cada época
#         losses.append(loss.item())

#         # Imprimir información de progreso
#         if (epoch + 1) % 10 == 0:
#             print(f"Epoch {epoch + 1}/{num_epochs}, Loss = {loss.item():.4f}")

#     # Crear directorio para guardar los resultados
#     os.makedirs("results", exist_ok=True)

#     # Calcular la solución exacta en una malla fina para hacer la comparación
#     x_fine = torch.linspace(0, Lx, 201)
#     y_fine = torch.linspace(0, Ly, 201)
#     t_fine = torch.linspace(0, nt, 201)
#     x_fine, y_fine, t_fine = torch.meshgrid(x_fine, y_fine, t_fine)
#     u_exact = u_exact_fn(x_fine, y_fine, t_fine)

#     # Guardar la solución obtenida por la red neuronal en una malla fina
#     with torch.no_grad():
#         u_pred = model(torch.cat([x_fine.reshape(-1, 1), y_fine.reshape(-1, 1), t_fine.reshape(-1, 1)], dim=1)).reshape(201, 201, 201).cpu().numpy()
#     np.save("results/u_pred.npy", u_pred)

In [23]:
# Definir los parámetros del problema
Lx = 1.0
Ly = 1.0
nx = 51
ny = 51
nt = 20
alpha = 0.1

# Definir la condición inicial
u0 = np.zeros((nx, ny))
u0[10:20, 10:20] = 1.0

# Definir el tiempo final y el número de épocas de entrenamiento
T = 0.5
num_epochs = 500

# Entrenar el modelo
train(model, optimizer, num_epochs, u0, Lx, Ly, nt, alpha)

TypeError: repeat(): argument 'repeats' must be tuple of SymInts, but found element of type float at pos 2